In [68]:
import time
from sys import maxsize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from sklearn.linear_model import Lasso
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from IPython.display import display, HTML

# Stochastic Gradient Optimization

In [70]:
"""
Sign of learningRate determines whether to use gradient ascent/descent:
Negative learningRates = descent, positive = ascent
"""
def SGO(x, y, seed, gradientFunc, learningRate, error, maxIter):
    x = np.array(x)
    y = np.array(y)

    # Reshape 1-D arrays to column format
    # if len(x.shape) == 1:
    #     x = x.reshape(-1, 1)

    N = len(x)
    currentError = maxsize
    lastError = 0
    beta = np.array(seed)
    i = 0

    while i < maxIter:
        sqErrGradient = gradientFunc(beta, x, y)
#         yPredicted = x.dot(beta)
#         sqErrGradient = np.array(np.dot(x.T, (yPredicted - y)) / N)
        beta += learningRate * sqErrGradient
#         currentError = np.sum(np.square(y - yPredicted)) / N
        currentError = np.sum(sqErrGradient)
#         print(currentError)

        if abs(lastError - currentError) < error:
            break
        lastError = currentError
        i += 1

    return beta

# SVM

In [207]:
class MySVM:
    def __init__(self, C, learningRate=1e-3, error=1e-2, maxIter=1000,
                 kernel='linear', method='primal'):
        if kernel == 'linear':
            self.kernel =  self._linearKernel
            
        self.method = method
        self.learningRate = learningRate
        self.error = error
        self.maxIter = maxIter
        self.C = C
    
    def fit(self, x, y):
        xCopy = self._removeIntercept(x)
        seed = np.zeros(x.shape[1] + 1)
        
        if self.method == 'primal':
            self.weights = SGO(xCopy, y, seed, self._svmPrimalGradient,
                               -self.learningRate, self.error, self.maxIter)
            
        else:
            self.weights = SGO(x, y, seed, self.method,
                    self.learningRate, self.error, self.maxIter)
    
    def predict(self, x):
        return np.sign(self.weights.T.dot(np.insert(x, len(x) - 1, 1)))
    
    # Return (accuracy, precision, recall, F1) tuple
    def score(self, x, yGold):
        TPs = []
        TNs = []
        FPs = []
        FNs = []
        predictions = [self.predict(xVal) for xVal in x]
        correct = np.sum([1 for xVal, yVal in zip(x, yGold)
                       if self.predict(xVal) == yVal])
        
        for classVal in [-1, 1]:
            TP = TN = FP = FN = 0
            for i, truth in enumerate(yGold):
                prediction = predictions[i]
                if prediction == truth == classVal:
                    TP += 1
                elif prediction == truth and truth != classVal:
                    TN += 1
                elif prediction != truth and truth == classVal:
                    FN += 1
                else:
                    FP += 1
            TPs.append(TP)
            TNs.append(TN)
            FPs.append(FP)
            FNs.append(FN)
        
        accuracy = sum(TPs) / len(yGold)
        precision = sum(TPs) / (2 * (sum(TPs) + sum(FPs)))
        recall = sum(TPs) / (2 * (sum(TPs) + sum(FNs)))
        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0
                 
        return accuracy, precision, recall, f1
    
    def _svmPrimalGradient(self, weights, x, y):
        self.supportVectorIndices = [i for i in range(len(x))
                                    if y[i] * x[i].T.dot(weights) < 1]
        signedSupportVecs = np.array([y[i] * x[i] for i in self.supportVectorIndices])
        return weights - self.C * np.sum(signedSupportVecs, axis=0) \
            - self.C * np.sum(y[i] for i in self.supportVectorIndices)
    
    def _svmDualGradient(self, alpha, x, y):
        gradShape = x.shape[0]
#         if len(x.shape) == 1:
#             gradShape = x.shape[0]
#         else:
#             gradShape = x.shape[1]

        grad = np.ndarray(shape=gradShape, dtype=float)

        for i in range(len(grad)):
            grad[i] = 1 - y[i] * np.sum(alpha[i] * x[i].dot(x.T))
        return grad
    
    def _removeIntercept(self, x):
        xCopy = np.array(x)
        
        if len(xCopy.shape) == 1:
            xCopy = xCopy.reshape(-1, 1)
#         print('X shape:', x.shape)
#         print('Ones shape:', np.ones(x.shape[0]).shape)
        xCopy = np.insert(xCopy, 0, np.ones(x.shape[0]), axis=1)
        
        return xCopy
    
    def _linearKernel():
        pass

# Predicting Malignancy of Breast Cancer Cases
## Source: [UCI ML Repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)

In [208]:
breastCancerDf = pd.read_csv('data/breast-cancer-wisconsin.csv')
breastCancerDf.loc[breastCancerDf['Class'] == 2, 'Class'] = 1
breastCancerDf.loc[breastCancerDf['Class'] == 4, 'Class'] = -1
# breastCancerDf.drop('BareNuclei', axis=1)
breastCancerDf.drop(breastCancerDf[breastCancerDf['BareNuclei'] == '?'].index, inplace=True)

yColumn = 'Class'
xColumns = [col for col in breastCancerDf.columns
            if col != 'ID' and col != yColumn
           and col != 'BareNuclei']
display(breastCancerDf[xColumns + [yColumn]].head())

print('% malignant', len(breastCancerDf['Class'].loc[breastCancerDf['Class'] == 1]) / len(breastCancerDf['Class']))

# Split into test and training sets
xTrain, xTest, yTrain, yTest = train_test_split(breastCancerDf[xColumns].as_matrix(),
                                               breastCancerDf[yColumn].as_matrix(),
                                               test_size=1/3, random_state=int(time.time()))
# np.random.seed(524)
# trainProportion = 0.8
# trainMask = np.random.rand(len(breastCancerDf)) < trainProportion
# cancerTrainingDf = breastCancerDf[trainMask]
# cancerTestDf = breastCancerDf[~trainMask].reset_index()
# print('Total # cancer samples: {}, training samples: {}, test samples: {}'.format(
#     len(breastCancerDf), len(cancerTrainingDf), len(cancerTestDf)))

,ClumpThickness,CellSizeUniformity,CellShapeUniformity,MarginalAdhesion,SingleEpithelialCellSize,BlandChromatin,NormalNucleoli,Mitoses,Class
0,5,1,1,1,2,3,1,1,1
1,5,4,4,5,7,3,2,1,1
2,3,1,1,1,2,3,1,1,1
3,6,8,8,1,3,3,7,1,1
4,4,1,1,3,2,3,1,1,1


% malignant 0.6500732064421669


In [209]:
classifier = MySVM(C=1, learningRate=1e-3, error=1e-3)
classifier.fit(xTrain, yTrain)
supportVecs = classifier.supportVectorIndices
print('Support vectors len={}: {}'.format(len(supportVecs), supportVecs))
accuracy, precision, recall, f1 = classifier.score(xTest, yTest)
print('Accuracy: {:.4f}, precision: {:.4f}, recall: {:.4f}, F1: {:.4f}'.format(
        accuracy, precision, recall, f1))

Support vectors len=28: [2, 76, 100, 120, 130, 132, 144, 172, 186, 187, 209, 232, 238, 275, 303, 323, 335, 339, 378, 384, 388, 400, 404, 405, 415, 421, 426, 440]
Accuracy: 0.6404, precision: 0.3202, recall: 0.3202, F1: 0.3202


## Sklearn Linear SVC for Comparison

In [131]:
testSVM = svm.SVC(kernel='linear')
testSVM.fit(xTrain, yTrain)
print('Coefficients: {}, beta: {}'.format(testSVM.coef_, testSVM.intercept_))
print('Support vectors len={}: {}'.format(len(testSVM.support_), testSVM.support_))
scores = cross_val_score(testSVM, breastCancerDf[xColumns].as_matrix(),
                        breastCancerDf[yColumn].as_matrix(),
                        cv=5)
predicted = testSVM.predict(xTest)
precision, recall, f1, support = precision_recall_fscore_support(yTest, predicted,
                                                                average='binary')
print('Sklearn SVM Accuracy:', scores.mean())
print('Precision={}, recall={}, F1={}'.format(precision, recall, f1))

Coefficients: [[-0.21728532 -0.07919049 -0.19345441 -0.16351294 -0.21623691 -0.21511535
  -0.12487737 -0.09989353]], beta: [ 4.42910256]
Support vectors len=43: [ 38  51  58  66  67 133 134 168 180 197 210 248 270 278 290 316 347 353
 354 370 388 441 453   5  14  49  57 117 122 181 185 220 259 288 301 327
 349 355 359 365 371 411 425]
Sklearn SVM Accuracy: 0.964898621249
Precision=0.98125, recall=0.9691358024691358, F1=0.9751552795031055
